In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm

from glob import glob
import pathlib
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Flatten, Dropout

import warnings
warnings.filterwarnings("ignore")

In [ ]:
image_path = "../input/game-of-deep-learning-ship-datasets/train/images/"
shape = (64, 64, 1)
mapping = {
    0: 'Cargo', 
    1: 'Military', 
    2: 'Carrier', 
    3: 'Cruise', 
    4: 'Tankers'
}
batch_size = 24

In [ ]:
train_csv_file = "../input/game-of-deep-learning-ship-datasets/train/train.csv"
train_csv = pd.read_csv(train_csv_file)
train_csv.sample(5)

In [ ]:
train_images = []
for i in tqdm(range(train_csv.shape[0])):
    img = image.load_img(image_path + train_csv['image'][i], target_size=shape, grayscale=True)
    img = image.img_to_array(img)
    img = img.astype(np.float32) / 255.0
    train_images.append(img)
X = np.array(train_images)

In [ ]:
y_val = train_csv['category'].subtract(1).values
y = to_categorical(y_val, num_classes=len(mapping))

## Validating the images

In [ ]:
Image.open(image_path + train_csv['image'][3254])

In [ ]:
plt.imshow(X[3254])
plt.show()

In [ ]:
val = train_csv.loc[:, "category"].value_counts()
val = val.rename(index=mapping)
val.plot.pie(
    autopct='%.2f'
)

## Model Building: ResNet50

In [ ]:
base_model = ResNet50(
    include_top=False, weights=None, input_shape=shape
)
layer = base_model.output
layer = Flatten()(layer)
layer = Dense(512, activation='tanh')(layer)
# layer = Dropout(0.3)(layer)
layer = Dense(128, activation='tanh')(layer)
# layer = Dropout(0.3)(layer)
layer = Dense(len(mapping), activation='softmax')(layer)
model = Model(inputs = base_model.input, outputs = layer)
model.summary()

In [ ]:
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
model.compile(
    optimizer='adam', 
    loss='binary_crossentropy',
    metrics=['accuracy']
)

## Performing the test train split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

## Perform Image Augmentation

In [ ]:
args = dict(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)
datagen = image.ImageDataGenerator(**args)

In [ ]:
train_dataset = datagen.flow(X_train, y_train, batch_size=batch_size)
test_dataset = datagen.flow(X_test, y_test, batch_size=batch_size)

In [ ]:
step_size_train=train_dataset.n//train_dataset.batch_size
step_size_val=test_dataset.n//test_dataset.batch_size

In [ ]:
history = model.fit(
    train_dataset,
    steps_per_epoch=step_size_train,
    epochs=50,
    validation_data=test_dataset,
    validation_steps=step_size_val
)

In [ ]:
plt.plot(history.history["loss"], label="train_loss")
plt.plot(history.history["val_loss"], label="val_loss")
plt.plot(history.history["acc"], label="train_acc")
plt.plot(history.history["val_acc"], label="val_acc")
plt.legend()
plt.show()